In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import mysql.connector
from PIL import Image, ImageTk

# Connect to MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="mc110322",
    database="Animation"
)
cursor = db.cursor()

# Function to execute SQL query and return results
def execute_query(query, params=None):
    try:
        cursor.execute(query, params or ())
        return cursor.fetchall(), cursor.column_names
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", str(err))
        return [], []

# Function to create a separate window for results
def create_result_window(root, column_names):
    tree = ttk.Treeview(root, columns=column_names, show="headings")
    for col in column_names:
        tree.heading(col, text=col)
    tree.pack(fill=tk.BOTH, expand=True)
    return tree

# Function to display results in a Treeview
def display_result(tree, result, column_names):
    tree.delete(*tree.get_children())
    tree['columns'] = column_names
    for col in column_names:
        tree.heading(col, text=col)
        tree.column(col, width=100, anchor=tk.CENTER)
    for row in result:
        tree.insert('', 'end', values=row)

# Function to open questions window
def open_questions_window():
    questions_window = tk.Toplevel(root)
    questions_window.title("Predefined Queries")
    questions = [
    ("Top 20 highest-rated movies", "SELECT title, voteaverage FROM movies ORDER BY voteaverage DESC LIMIT 20", ["Title", "Vote Average"]),
    ("Number of movies released in each year", "SELECT YEAR(release_date) AS Year, COUNT(*) AS MoviesReleased FROM movies GROUP BY YEAR(release_date) order by Year", ["Year", "Movies Released"]),
    ("Top 10 movies with the longest runtime", "SELECT title, runtime FROM movies ORDER BY runtime DESC LIMIT 10", ["Title", "Runtime"]),
    ("Movies with revenue greater than $1 billion", "SELECT title, revenue FROM movies WHERE revenue > 1000000000", ["Title", "Revenue"]),
    ("Number of movies in each genre", "SELECT genres, COUNT(*) AS NumMovies FROM movies GROUP BY genres", ["Genre", "Number of Movies"]),
    ("Average vote count per movie", "SELECT AVG(votecount) AS AverageVoteCount FROM movies", ["Average Vote Count"]),
    ("Movies with runtime greater than 180 minutes", "SELECT title, runtime FROM movies WHERE runtime > 180", ["Title", "Runtime"]),
    ("Movies released after 2015 with vote average greater than 8", "SELECT title, voteaverage, release_date FROM movies WHERE YEAR(release_date) > 2015 AND voteaverage > 8", ["Title", "Vote Average", "Release Date"]),
    ("Movies with a tagline", "SELECT title, tagline FROM movies WHERE tagline IS NOT NULL AND tagline != ''", ["Title", "Tagline"]),
    ("Movies with poster path", "SELECT title, posterpath FROM movies WHERE posterpath IS NOT NULL AND posterpath != ''", ["Title", "Poster Path"]),
    ("Movies Released During COVID-19 in 2020", "SELECT title, release_date, genres, revenue FROM movies WHERE YEAR(release_date) = 2020 ORDER BY release_date",["Title", "Release Date", "Genres", "Revenue"]),
    ("Most Active Production Companies", "SELECT productioncompanies, COUNT(*) AS MoviesProduced FROM movies GROUP BY productioncompanies ORDER BY MoviesProduced DESC LIMIT 10", ["Production Company", "Movies Produced"]),
    ("Movies produced by Disney", "SELECT title, productioncompanies FROM movies WHERE productioncompanies LIKE '%Disney%'", ["Title", "Production Company"]),
    ("Comedy movies with runtime less than 120 minutes", "SELECT title, genres, runtime FROM movies WHERE genres = 'Comedy' AND runtime < 120", ["Title", "Genre", "Runtime"]),
    ("Top Rated Movies for Children", "SELECT title, voteaverage FROM movies WHERE genres LIKE '%Family%' AND voteaverage > 8 ORDER BY voteaverage DESC LIMIT 10", ["Title", "Vote Average"]),
    ("Top Grossing Movies that are Not in English", "SELECT title, revenue FROM movies WHERE spokenlanguages NOT LIKE '%English%' ORDER BY revenue DESC LIMIT 10", ["Title", "Revenue"]),
    ("Impact of Movie Length on Popularity", "SELECT title, runtime, popularity FROM movies ORDER BY popularity DESC, runtime DESC LIMIT 10", ["Title", "Runtime", "Popularity"]),
     ("Movies with the Best ROI", "SELECT title, budget, revenue, ((revenue - budget)/budget * 100) AS ROI FROM movies WHERE budget > 0 AND revenue > 0 ORDER BY ROI DESC LIMIT 10", ["Title", "Budget", "Revenue", "ROI"]),
    ("Movies with Overviews Longer Than Average", "SELECT title, LENGTH(overview) AS OverviewLength, (SELECT AVG(LENGTH(overview)) FROM movies) AS AverageLength FROM movies HAVING OverviewLength > AverageLength ORDER BY OverviewLength DESC LIMIT 10", ["Title", "Overview Length", "Average Length"]),
    ("Summer vs Winter Revenue Comparison", "SELECT 'Summer' AS Season, SUM(revenue) AS TotalRevenue FROM movies WHERE MONTH(release_date) IN (6, 7, 8) UNION ALL SELECT 'Winter' AS Season, SUM(revenue) FROM movies WHERE MONTH(release_date) IN (12, 1, 2)", ["Season", "Total Revenue"]),
    ("Monthly Movie Releases", "SELECT MONTH(release_date) AS Month, COUNT(*) AS MovieCount FROM movies GROUP BY Month ORDER BY MovieCount DESC", ["Month", "Movie Count"]),
    ("Top Grossing Months", "SELECT MONTH(release_date) AS Month, SUM(revenue) AS TotalRevenue FROM movies GROUP BY Month ORDER BY TotalRevenue DESC", ["Month", "Total Revenue"]),
    ("Seasonal Analysis of Movie Ratings", "SELECT CASE WHEN MONTH(release_date) IN (3, 4, 5) THEN 'Spring' WHEN MONTH(release_date) IN (6, 7, 8) THEN 'Summer' WHEN MONTH(release_date) IN (9, 10, 11) THEN 'Fall' ELSE 'Winter' END AS Season, AVG(voteaverage) AS AverageRating FROM movies GROUP BY Season ORDER BY AverageRating DESC", ["Season", "Average Rating"]),
    ("Best Performing Months for Animation", "SELECT MONTH(release_date) AS Month, AVG(revenue) AS AverageRevenue FROM movies WHERE genres LIKE '%Animation%' GROUP BY Month ORDER BY AverageRevenue DESC", ["Month", "Average Revenue"]),
    ("Impact of Holiday Seasons on Revenue", "SELECT CASE WHEN MONTH(release_date) = 12 OR MONTH(release_date) = 1 THEN 'Holiday Season' ELSE 'Regular Season' END AS Season, AVG(revenue) AS AverageRevenue FROM movies GROUP BY Season", ["Season", "Average Revenue"]),
    ("Analysis of Viewer Engagement by Month", "SELECT MONTH(release_date) AS Month, AVG(popularity) AS AveragePopularity FROM movies GROUP BY Month ORDER BY AveragePopularity DESC", ["Month", "Average Popularity"])
]
    question_combobox = ttk.Combobox(questions_window, values=[q[0] for q in questions], state="readonly", width=50)
    question_combobox.pack(pady=10)
    result_tree = create_result_window(questions_window, ["Title", "Vote Average"])
    def display_answer():
        selected_index = question_combobox.current()
        if selected_index >= 0:
            query = questions[selected_index][1]
            result, column_names = execute_query(query)
            display_result(result_tree, result, column_names)
    display_button = tk.Button(questions_window, text="Get your answers", command=display_answer)
    display_button.pack()

# Function to open filter window
def open_filter_window():
    filter_window = tk.Toplevel(root)
    filter_window.title("Filter & Search Movies")
    genre_var = tk.StringVar(value="No Filter")
    language_var = tk.StringVar(value="No Filter")
    runtime_var = tk.StringVar(value="No Filter")
    adult_var = tk.StringVar(value="No Filter")
    vote_average_min_var = tk.DoubleVar(value=0.0)
    vote_average_max_var = tk.DoubleVar(value=10.0)
    search_var = tk.StringVar()
    genre_label = tk.Label(filter_window, text="Genre:")
    genre_label.pack()
    genres = ["No Filter", "Action", "Adventure", "Animation", "Biography", "Comedy", "Crime", "Documentary", "Drama", "Family", "Fantasy", "History", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Sport", "Thriller", "War", "Western"]
    genre_options = ttk.Combobox(filter_window, textvariable=genre_var, values=genres, state="readonly")
    genre_options.pack()
    language_label = tk.Label(filter_window, text="Language:")
    language_label.pack()
    languages = ["No Filter", "English", "French", "Spanish", "German", "Chinese", "Japanese", "Korean", "Italian", "Russian", "Portuguese", "Arabic", "Hindi", "Bengali"]
    language_options = ttk.Combobox(filter_window, textvariable=language_var, values=languages, state="readonly")
    language_options.pack()
    tk.Label(filter_window, text="Runtime:").pack()
    runtime_options = ttk.Combobox(filter_window, textvariable=runtime_var, values=["No Filter", "<90", "90-120", ">120"], state="readonly")
    runtime_options.pack()
    tk.Label(filter_window, text="Adult Content:").pack()
    adult_options = ttk.Combobox(filter_window, textvariable=adult_var, values=["No Filter", "Yes", "No"], state="readonly")
    adult_options.pack()
    tk.Label(filter_window, text="Vote Average (Min):").pack()
    vote_average_min_entry = ttk.Entry(filter_window, textvariable=vote_average_min_var)
    vote_average_min_entry.pack()
    tk.Label(filter_window, text="Vote Average (Max):").pack()
    vote_average_max_entry = ttk.Entry(filter_window, textvariable=vote_average_max_var)
    vote_average_max_entry.pack()
    search_label = tk.Label(filter_window, text="Search within filter:")
    search_label.pack()
    search_entry = ttk.Entry(filter_window, textvariable=search_var)
    search_entry.pack()
    result_tree = create_result_window(filter_window, ["Title", "Genres", "Spoken Languages", "Runtime", "Vote Average"])
    def apply_filters():
        genre = genre_var.get() if genre_var.get() != "No Filter" else "%"
        language = language_var.get() if language_var.get() != "No Filter" else "%"
        runtime = runtime_var.get()
        if runtime == "No Filter":
            runtime_condition = "TRUE"
        elif runtime == "<90":
            runtime_condition = "runtime < 90"
        elif runtime == "90-120":
            runtime_condition = "runtime >= 90 AND runtime <= 120"
        else:
            runtime_condition = "runtime > 120"
        
        adult_condition = "TRUE" if adult_var.get() == "No Filter" else f"adult = '{'TRUE' if adult_var.get() == 'Yes' else 'FALSE'}'"
        
        vote_average_min = vote_average_min_var.get()
        vote_average_max = vote_average_max_var.get()
        search_text = f"%{search_var.get()}%"
        query = f"""SELECT title, genres, spokenlanguages, runtime, voteaverage FROM movies 
                    WHERE genres LIKE %s AND spokenlanguages LIKE %s AND {runtime_condition} AND {adult_condition} AND 
                    voteaverage >= %s AND voteaverage <= %s AND title LIKE %s"""
        result, column_names = execute_query(query, (genre, language, vote_average_min, vote_average_max, search_text))
        display_result(result_tree, result, column_names)
        
    apply_button = tk.Button(filter_window, text="Apply Filters and Search", command=apply_filters)
    apply_button.pack(side=tk.LEFT, padx=10, pady=10)

    # Adding a clear filter button
    def clear_filters():
        genre_var.set("No Filter")
        language_var.set("No Filter")
        runtime_var.set("No Filter")
        adult_var.set("No Filter")
        vote_average_min_var.set(0.0)
        vote_average_max_var.set(10.0)
        search_var.set("")
        # Clear the results treeview
        display_result(result_tree, [], [])

    clear_button = tk.Button(filter_window, text="Clear Filters", command=clear_filters)
    clear_button.pack(side=tk.RIGHT, padx=10, pady=10)

# Function to search movies by title
def search_by_title():
    search_window = tk.Toplevel(root)
    search_window.title("Search Movies by Title")
    search_label = tk.Label(search_window, text="Search Movies by Title:")
    search_label.pack()
    search_var = tk.StringVar()
    search_entry = ttk.Entry(search_window, textvariable=search_var)
    search_entry.pack()
    result_tree = create_result_window(search_window, ["Title", "Genres", "Spoken Languages", "Runtime", "Vote Average"])
    def apply_search():
        search_text = f"%{search_var.get()}%"
        query = "SELECT title, genres, spokenlanguages, runtime, voteaverage FROM movies WHERE title LIKE %s"
        result, column_names = execute_query(query, (search_text,))
        display_result(result_tree, result, column_names)
    search_button = tk.Button(search_window, text="Search", command=apply_search)
    search_button.pack()

# Main application
root = tk.Tk()
root.title("Animated Movies Interface")
try:
    welcome_image = Image.open("/Users/meharchaudhry/Desktop/AnimatedMovies.jpeg")
    welcome_photo = ImageTk.PhotoImage(welcome_image.resize((800, 400)))
    welcome_label = tk.Label(root, image=welcome_photo)
    welcome_label.image = welcome_photo
    welcome_label.pack(pady=20)
except Exception as e:
    messagebox.showerror("Image Load Error", str(e))

questions_button = tk.Button(root, text="Get Your Answers to Questions", command=open_questions_window)
questions_button.pack(fill=tk.X, padx=50, pady=10)

filter_button = tk.Button(root, text="Filter and Find Your Favorite Movie", command=open_filter_window)
filter_button.pack(fill=tk.X, padx=50, pady=10)

search_button = tk.Button(root, text="Search Movies by Title", command=search_by_title)
search_button.pack(fill=tk.X, padx=50, pady=10)

root.mainloop()